In [ ]:
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
sst_data = [[20,24,21,18],[21,23,26,22],[19,23,25,21]]
lat_data, lon_data = [30,35,40], [-145, -140,-135,-130 ]
da = xr.DataArray(sst_data, name='SST', coords={'lat':lat_data,'lon':lon_data}, dims=('lat','lon'))


In [ ]:
ds = xr.Dataset({'SST1':da,'SST2':da+0.5,'SST3':da-0.5})


In [ ]:
#url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/2011/305/20111101120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_sst = xr.open_dataset(url)
ds_sst

In [ ]:
ds_sst.analysed_sst

In [ ]:
ds_sst.analysed_sst.attrs

In [ ]:
ds_sst.analysed_sst.values

In [ ]:
type(ds_sst.analysed_sst.values)

In [ ]:
ds_sst.analysed_sst.dims

In [ ]:
ds_sst.lat.size

In [ ]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ccmp/L3.0/flk/2011/11/analysis_20111101_v11l30flk.nc.gz'
ds_ccmp = xr.open_dataset(url)
ds_ccmp

In [ ]:
ds_ccmp.coords['lon'] = np.mod(ds_ccmp.coords['lon'] + 180,360) - 180
ds_ccmp = ds_ccmp.sortby(ds_ccmp.lon)
ds_ccmp

In [ ]:
ds_ccmp['wind_speed'] = np.sqrt(ds_ccmp.uwnd**2 + ds_ccmp.vwnd**2)
day_ave = ds_ccmp.mean(dim='time')
day_ave.wind_speed.plot();

In [ ]:
ds_sst.analysed_sst[0, :, 900]  # note that the attributes, coordinates are preserved

In [ ]:
ds_sst.analysed_sst[0, :, 900].plot();

In [ ]:
ds_sst.analysed_sst.isel(time=0).plot();  # like above, but with a dimension name this time

In [ ]:
ds_sst.analysed_sst.sel(lon=0).plot(); 

In [ ]:
ds_sst.analysed_sst.sel(time='2011-11-01').plot();  # or look at a single timestep

In [ ]:
subset = day_ave.sel(lat=slice(20,50),lon=slice(-140,-110))
subset.wind_speed.plot()

In [ ]:
ds_sst.analysed_sst.sel(lat=10, method='nearest', tolerance=5).plot();

In [ ]:
ds_ccmp_masked = day_ave.where(day_ave.nobs>0)
ds_ccmp_masked.wind_speed.plot()

In [ ]:
ds_ccmp_interp = ds_ccmp.interp_like(ds_sst)
ds_ccmp_interp

In [ ]:
masked = ds_sst.where(ds_ccmp_interp.wind_speed<2)
masked.analysed_sst.plot()

In [ ]:
masked = ds_ccmp_interp.where(ds_sst.mask==1)
masked.wind_speed.plot()

In [ ]:
fig, axes = plt.subplots(ncols=2)
ds_ccmp_masked.wind_speed.plot.hist(bins=20,ax=axes[0],density = True)
ds_ccmp.wind_speed.plot.hist(bins=20,ax=axes[1],density = True,histtype='step')

In [ ]:
ds_ccmp.wind_speed.plot.contourf(x='lon', y='lat', col='time', col_wrap=2, levels=22);

In [ ]:
import cartopy.crs as ccrs

In [ ]:
f = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))
day_ave.wind_speed.plot(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines(); 

In [ ]:
subset = ds_ccmp.sel(lat=slice(20,50),lon=slice(-170,-110)).mean('time')
f = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))
subset.wind_speed.plot(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines(); 


In [ ]:
subset = ds_ccmp.sel(lat=slice(20,50),lon=slice(-170,-110)).mean('time')
f = plt.figure(figsize=(8, 4))
ax = plt.axes(projection=ccrs.PlateCarree())
subset.wind_speed.plot(ax=ax, transform=ccrs.PlateCarree())
ax.coastlines(); ax.gridlines(); 
ax.set_xlim(-170,-110); ax.set_ylim(20,50)


In [ ]:
p = ds_ccmp.wind_speed.plot(x='lon', y='lat', col='time', transform=ccrs.PlateCarree(),
                  subplot_kws={'projection': ccrs.Orthographic(-80, 35)})
for ax in p.axes.flat:
    ax.coastlines()
#fig_fname='./images/ccmp_6hourly.png'
#p.fig.savefig(fig_fname, transparent=False, format='png')

In [ ]:
from glob import glob
filepath = './data/avhrr_oi/*.nc'
files = glob(filepath,recursive=True)
files[:10]

In [ ]:
ds_sst = xr.open_mfdataset(files,coords='minimal')
ds_sst

In [ ]:
ds_sst.coords['lon'] = np.mod(ds_sst.coords['lon'] + 180,360) - 180
ds_sst = ds_sst.sortby(ds_sst.lon)
ds_sst = ds_sst.isel(zlev=0)
ds_sst

In [ ]:
ds_sst.time

In [ ]:
ds_sst.nbytes / 1e9  

In [ ]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds_usv = xr.open_dataset(url)
ds_usv

In [ ]:
ds_usv = ds_usv.isel(trajectory=0).swap_dims({'obs':'time'}).rename({'longitude':'lon','latitude':'lat'})
ds_usv_subset = ds_usv.sel(time=slice('2018-04-12T02','2018-06-10T18')) 

In [ ]:
#Step 1 from above
print('min max lat lon:', ds_usv_subset.lon.min().data,ds_usv_subset.lon.max().data,ds_usv_subset.lat.min().data,ds_usv_subset.lat.max().data)
subset = ds_sst.sel(lon=slice(ds_usv_subset.lon.min().data,ds_usv_subset.lon.max().data),
                    lat=slice(ds_usv_subset.lat.min().data,ds_usv_subset.lat.max().data))
#Step 2 from above
subset.load()

In [ ]:
ds_collocated = subset.interp(lat=ds_usv_subset.lat,lon=ds_usv_subset.lon,time=ds_usv_subset.time,method='linear')

In [ ]:
ds_collocated_nearest = subset.interp(lat=ds_usv_subset.lat,lon=ds_usv_subset.lon,time=ds_usv_subset.time,method='nearest')

In [ ]:
dif = ds_collocated_nearest.sst-ds_usv.TEMP_CTD_MEAN
print('mean difference = ',dif.mean().data)
print('STD = ',dif.std().data)

In [ ]:
#ds_collocated_nearest.to_netcdf('./data/new file.nc')